In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report 
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from hyperopt import hp 
from hyperopt import fmin, tpe, Trials 
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
pca_train = pd.read_csv('PCA_train_data.csv')
pca_test = pd.read_csv('PCA_test_data.csv')
tree_train = pd.read_csv('Tree_train_data.csv')
tree_test= pd.read_csv('Tree_test_data.csv')
trainset= pd.read_csv("train_dataset.csv")
testset= pd.read_csv('test_dataset.csv')

In [4]:
trainset.head()

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,attack,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0,0,0,normal,0.0,0.0,1.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0,0,0,neptune,0.0,1.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00


In [6]:
pca_train.head()


,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,attack,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,...,is_attack,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9
0,0,0,0,0,0,0,normal,0.0,1.0,0.0,...,0.0,-0.940599,-0.473843,0.719614,0.117456,-0.357645,-0.162415,-0.191790,-0.219253,-0.014388
1,0,0,0,0,0,0,normal,0.0,0.0,1.0,...,0.0,-0.728731,-0.240017,1.426927,0.096360,0.159278,-0.026683,0.227241,0.402077,-0.355985
2,0,0,0,0,0,0,neptune,0.0,1.0,0.0,...,1.0,3.035355,0.142109,-0.102676,-0.026129,-0.011964,0.012623,-0.021454,0.066945,-0.089175
3,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.0,-0.802288,-0.822607,-0.560944,0.151033,-0.207357,-0.232259,0.136882,0.050881,0.049239
4,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.0,-1.134026,-0.812825,-0.190227,-0.525232,0.102108,0.061269,-0.123393,0.092231,-0.040100


In [5]:
X_train = trainset.drop(columns=['is_attack', 'attack_category',"attack"])
y_train = trainset["attack_category"]
X_test = testset.drop(columns=['is_attack', 'attack_category',"attack"])
y_test= testset["attack_category"]


In [6]:
X_pca_train = pca_train.drop(columns=['is_attack', 'attack_category',"attack"])
y_pca_train = pca_train["attack_category"]
X_pca_test = pca_test.drop(columns=['is_attack', 'attack_category',"attack"])
y_pca_test= pca_test["attack_category"]


In [7]:
X_tree_train = tree_train.drop(columns=['is_attack', 'attack_category',"attack"])
y_tree_train = tree_train["attack_category"]
X_tree_test = tree_test.drop(columns=['is_attack', 'attack_category',"attack"])
y_tree_test= tree_test["attack_category"]

In [8]:
trainset

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,attack,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0,0,0,normal,0.0,0.0,1.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0,0,0,neptune,0.0,1.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,neptune,0.0,1.0,0.0,...,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,4.00,0.00
125969,0,0,0,0,0,0,normal,0.0,0.0,1.0,...,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00
125970,0,1,0,0,0,0,normal,0.0,1.0,0.0,...,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.01
125971,0,0,0,0,0,0,neptune,0.0,1.0,0.0,...,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,4.00,0.00


In [9]:
X_tree_train

,logged_in,root_shell,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,4.00,0.00
125969,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00
125970,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.01
125971,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,4.00,0.00


In [10]:
X_pca_train

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_OTH,...,flag_SH,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9
0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.940599,-0.473843,0.719614,0.117456,-0.357645,-0.162415,-0.191790,-0.219253,-0.014388
1,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-0.728731,-0.240017,1.426927,0.096360,0.159278,-0.026683,0.227241,0.402077,-0.355985
2,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.035355,0.142109,-0.102676,-0.026129,-0.011964,0.012623,-0.021454,0.066945,-0.089175
3,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.802288,-0.822607,-0.560944,0.151033,-0.207357,-0.232259,0.136882,0.050881,0.049239
4,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-1.134026,-0.812825,-0.190227,-0.525232,0.102108,0.061269,-0.123393,0.092231,-0.040100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.026111,0.134002,-0.102845,-0.034096,0.043781,-0.004933,-0.007050,-0.006698,0.012918
125969,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-1.119040,-0.792833,-0.115345,-0.513639,0.062364,-0.016443,-0.155788,0.095988,-0.082379
125970,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.211762,-0.391754,0.632769,-0.190737,-0.261550,-0.086665,-0.416727,-0.160747,0.057605
125971,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.059266,0.171533,0.003287,-0.001806,-0.018560,0.020528,-0.023829,-0.019866,-0.062215


## Testing Smote on PCA dataset

In [9]:
from imblearn.over_sampling import SMOTE
from collections import Counter


In [10]:
print("Before SMOTE:", Counter(y_pca_train))
sampling_strategy = {3: 10000, 4: 5000}

# Instantiating the SMOTE object
smote = SMOTE(random_state = 42, sampling_strategy = sampling_strategy)

# Applying SMOTE to generate synthetic samples
X_pca_resampled, y_pca_resampled = smote.fit_resample(X_pca_train, y_pca_train)

print("After SMOTE:", Counter(y_pca_resampled))


Before SMOTE: Counter({0: 67343, 1: 45927, 2: 11656, 3: 995, 4: 52})
After SMOTE: Counter({0: 67343, 1: 45927, 2: 11656, 3: 10000, 4: 5000})


In [11]:
def train_model(model, X, y):
    skf = StratifiedKFold(n_splits=5)

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for train_index, test_index in skf.split(X, y):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        model.fit(X_train, y_train)

        pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        precision = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted') 
        f1 = f1_score(y_test, pred, average='weighted') 

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    # print('{0} Accuracy: {1: .5f} Precision: {2: .5f} Recall: {3: .5f} F1_Score: {4: .5f}'.format(
    #     model.__class__.__name__, 
    #     np.mean(accuracy_scores),
    #     np.mean(precision_scores),
    #     np.mean(recall_scores),
    #     np.mean(f1_scores)))
    
    return -np.mean(f1_scores)

In [12]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print("Model: ", model.__class__.__name__)
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("####################################")
    
    # report = classification_report(y_test, y_pred)
    # print("Classification Report:\n", report)
    
    return -f1

In [13]:
def best_params(objective_func, search_space):
    trials = Trials() 

    best = fmin(fn=objective_func,
                space=search_space,
                algo=tpe.suggest,
                max_evals=20,
                trials=trials,
                rstate=np.random.default_rng(seed=30))

    print('Best Parameters:', best)
    return best 

In [40]:
clf =RandomForestClassifier(n_estimators=100, random_state=42)
evaluate(clf, X_pca_resampled,y_pca_resampled,X_pca_test,y_pca_test)

Model:  RandomForestClassifier
Accuracy:  0.7654364797728885
Precision:  0.8213316146576164
Recall:  0.7654364797728885
F1 Score:  0.7341330941956512
####################################


-0.7341330941956512

In [14]:
rf_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 1, 10, 1),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4, 6,8]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10,15]),
}

def objective_func_rf(search_space):
    # Create and train a Random Forest classifier with the specified hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        min_samples_split=search_space['min_samples_split'],
        min_samples_leaf=search_space['min_samples_leaf'],
        random_state=42
    )

 
    return train_model(rf_clf,X_pca_resampled,y_pca_resampled)
    
    
    
    


In [46]:
rf_best = best_params(objective_func_rf, rf_search_space)

100%|██████████| 50/50 [1:28:44<00:00, 106.49s/trial, best loss: -0.9751116241684874]
Best Parameters: {'max_depth': 10.0, 'min_samples_leaf': 0, 'min_samples_split': 2, 'n_estimators': 300.0}


In [47]:
clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=1,
    random_state=42 
)
evaluate(clf, X_pca_resampled,y_pca_resampled,X_pca_test,y_pca_test)

Model:  RandomForestClassifier
Accuracy:  0.7638839602555003
Precision:  0.8221904668952357
Recall:  0.7638839602555003
F1 Score:  0.7399908472235582
####################################


-0.7399908472235582

# RANDOMFOREST Classifier


In [15]:
# Default Model
clf =RandomForestClassifier(n_estimators=100, random_state=42)
evaluate(clf, X_train,y_train,X_test,y_test)


Model:  RandomForestClassifier
Accuracy:  0.7621540099361249
Precision:  0.8237763007549375
Recall:  0.7621540099361249
F1 Score:  0.7164674293999976
####################################


-0.7164674293999976

In [16]:
rf_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 1, 10, 1),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 4, 6, 8,10]),
    'min_samples_split': hp.choice('min_samples_split', [2, 15, 20,30]),
}

def objective_func_rf(search_space):
    # Create and train a Random Forest classifier with the specified hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        min_samples_split=search_space['min_samples_split'],
        min_samples_leaf=search_space['min_samples_leaf'],
        random_state=42
    )

 
    return train_model(rf_clf,X_train,y_train)

In [18]:
rf_best = best_params(objective_func_rf, rf_search_space)

100%|██████████| 20/20 [10:44<00:00, 32.23s/trial, best loss: -0.9849622700834164]
Best Parameters: {'max_depth': 8.0, 'min_samples_leaf': 3, 'min_samples_split': 1, 'n_estimators': 150.0}


In [69]:
clf = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=15,
    min_samples_leaf=8,
    random_state=42 
)
evaluate(clf, X_train,y_train,X_test,y_test)

Model:  RandomForestClassifier
Accuracy:  0.7559882895670689
Precision:  0.8211970429149197
Recall:  0.7559882895670689
F1 Score:  0.7170903547161447
####################################


-0.7170903547161447

### PCA Dataset

In [19]:
clf =RandomForestClassifier(n_estimators=100, random_state=42)
evaluate(clf, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  RandomForestClassifier
Accuracy:  0.764194464158978
Precision:  0.8209953081361437
Recall:  0.764194464158978
F1 Score:  0.7279026168987286
####################################


-0.7279026168987286

In [20]:
rf_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 1, 10, 1),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4, 6,8]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10,15]),
}

def objective_func_rf(search_space):
    # Create and train a Random Forest classifier with the specified hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        min_samples_split=search_space['min_samples_split'],
        min_samples_leaf=search_space['min_samples_leaf'],
        random_state=42
    )

 
    return train_model(rf_clf,X_pca_train,y_pca_train)
    

In [21]:
rf_best = best_params(objective_func_rf, rf_search_space)

100%|██████████| 20/20 [26:35<00:00, 79.75s/trial, best loss: -0.9758201385254532]
Best Parameters: {'max_depth': 8.0, 'min_samples_leaf': 3, 'min_samples_split': 1, 'n_estimators': 150.0}


In [77]:
# Hyperparameters after trial and error from HyperOpt Best Params
clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42
)
evaluate( clf, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  RandomForestClassifier
Accuracy:  0.7592707594038325
Precision:  0.8203990298424062
Recall:  0.7592707594038325
F1 Score:  0.7234885789382247
####################################


-0.7234885789382247

For Tree Feature Selection

In [22]:
clf =RandomForestClassifier(n_estimators=100, random_state=42)
evaluate( clf, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  RandomForestClassifier
Accuracy:  0.761444286728176
Precision:  0.8193291240070046
Recall:  0.761444286728176
F1 Score:  0.7157725235435279
####################################


-0.7157725235435279

In [30]:
rf_search_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 50),
    'max_depth': hp.quniform('max_depth', 1, 10, 1),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4, 6,8]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10,15]),
}

def objective_func_rf2(search_space):
    # Create and train a Random Forest classifier with the specified hyperparameters
    rf_clf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        min_samples_split=search_space['min_samples_split'],
        min_samples_leaf=search_space['min_samples_leaf'],
        random_state=42
    )

 
    return train_model(rf_clf,X_tree_train,y_tree_train)


    

In [53]:
rf2_best = best_params(objective_func_rf2, rf_search_space)

100%|██████████| 20/20 [11:29<00:00, 34.48s/trial, best loss: -0.989203199228635]
Best Parameters: {'max_depth': 8.0, 'min_samples_leaf': 3, 'min_samples_split': 1, 'n_estimators': 150.0}


In [71]:
# Hyperparameters after trial and error from HyperOpt Best Params
clf = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    min_samples_split=20,
    min_samples_leaf=8,
    random_state=42
)
evaluate( clf, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  RandomForestClassifier
Accuracy:  0.7555003548616039
Precision:  0.8208712695442787
Recall:  0.7555003548616039
F1 Score:  0.7175159890795794
####################################


-0.7175159890795794

# GBM Classifier

In [25]:
X_train

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_OTH,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,4.00,0.00
125969,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00
125970,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.01
125971,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,4.00,0.00


In [98]:
X_pca_train

,land,logged_in,root_shell,su_attempted,is_host_login,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_OTH,...,flag_SH,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9
0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.940599,-0.473843,0.719614,0.117456,-0.357645,-0.162415,-0.191790,-0.219253,-0.014388
1,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-0.728731,-0.240017,1.426927,0.096360,0.159278,-0.026683,0.227241,0.402077,-0.355985
2,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.035355,0.142109,-0.102676,-0.026129,-0.011964,0.012623,-0.021454,0.066945,-0.089175
3,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.802288,-0.822607,-0.560944,0.151033,-0.207357,-0.232259,0.136882,0.050881,0.049239
4,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-1.134026,-0.812825,-0.190227,-0.525232,0.102108,0.061269,-0.123393,0.092231,-0.040100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.026111,0.134002,-0.102845,-0.034096,0.043781,-0.004933,-0.007050,-0.006698,0.012918
125969,0,0,0,0,0,0,0.0,0.0,1.0,0.0,...,0.0,-1.119040,-0.792833,-0.115345,-0.513639,0.062364,-0.016443,-0.155788,0.095988,-0.082379
125970,0,1,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,-0.211762,-0.391754,0.632769,-0.190737,-0.261550,-0.086665,-0.416727,-0.160747,0.057605
125971,0,0,0,0,0,0,0.0,1.0,0.0,0.0,...,0.0,3.059266,0.171533,0.003287,-0.001806,-0.018560,0.020528,-0.023829,-0.019866,-0.062215


In [99]:
X_tree_train

,logged_in,root_shell,is_guest_login,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,...,diff_srv_rate_scaled,srv_diff_host_rate_scaled,dst_host_count_scaled,dst_host_srv_count_scaled,dst_host_same_srv_rate_scaled,dst_host_diff_srv_rate_scaled,dst_host_same_src_port_rate_scaled,dst_host_srv_diff_host_rate_scaled,serror_rate_combined,rerror_rate_combined
0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.588235,0.098039,0.17,0.03,0.17,0.00,0.00,0.05
1,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00
2,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,4.00,0.00
3,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.44,0.01
4,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,4.00,0.00
125969,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00
125970,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.01
125971,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,4.00,0.00


In [33]:
gbm = GradientBoostingClassifier(n_estimators=100,  random_state=42)
evaluate(gbm, X_train,y_train,X_test,y_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7678317955997161
Precision:  0.8243788697571973
Recall:  0.7678317955997161
F1 Score:  0.729153072234855
####################################


-0.729153072234855

In [37]:
# Define the search space for hyperparameters
gbm_search_space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400]),
    'max_depth': hp.choice('max_depth', [3, 5, 7, 9, 11]),
    'learning_rate': hp.choice('learning_rate', [0.01, 0.05, 0.1, 0.2, 0.3]),
    'subsample': hp.choice('subsample', [0.5, 0.7,0.9])
    
}

# Define the objective function for optimizing F1 score
def objective_func_gbm(params):
    gbm = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        random_state=42
    )
    

    return train_model(gbm,X_train,y_train)


In [27]:
gbm_best = best_params(objective_func_gbm, gbm_search_space)

100%|██████████| 20/20 [6:39:29<00:00, 1198.47s/trial, best loss: -0.9976002091942894]  
Best Parameters: {'learning_rate': 0.012374515507867188, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 3, 'n_estimators': 3, 'subsample': 0.6739557674630375}


In [38]:
gbm = GradientBoostingClassifier(n_estimators=400, learning_rate= 0.012374515507867188, max_depth=9,subsample= 0.6739557674630375, random_state=42)
evaluate(gbm, X_train,y_train,X_test,y_test)

Model:  GradientBoostingClassifier
Accuracy:  0.754347054648687
Precision:  0.8063306689532979
Recall:  0.754347054648687
F1 Score:  0.7068861090278146
####################################


-0.7068861090278146

### PCA Dataset

In [39]:
gbm = GradientBoostingClassifier(n_estimators=100, random_state=42)
evaluate(gbm, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7800301632363378
Precision:  0.8271996274325536
Recall:  0.7800301632363378
F1 Score:  0.7563681226678713
####################################


-0.7563681226678713

In [44]:
# Define the search space for hyperparameters
gbm_search_space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400]),
    'max_depth': hp.choice('max_depth', [3, 5, 7, 9]),
    'learning_rate': hp.choice('learning_rate', [0.01, 0.05, 0.1, 0.3]),
    
    
}

# Define the objective function for optimizing F1 score
def objective_func_gbm(params):
    gbm = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        
        random_state=42
    )
    

    return train_model(gbm,X_pca_train,y_pca_train)



In [ ]:
gbm_best = best_params(objective_func_gbm, gbm_search_space)

In [46]:
gbm = GradientBoostingClassifier(n_estimators=300, learning_rate= 0.1, max_depth=7,random_state=42)
evaluate(gbm, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7625088715400994
Precision:  0.7876217474537873
Recall:  0.7625088715400994
F1 Score:  0.7323141874884166
####################################


-0.7323141874884166

In [48]:
gbm = GradientBoostingClassifier(n_estimators=400, learning_rate= 0.01, max_depth=7,random_state=42)
evaluate(gbm, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7601579134137686
Precision:  0.7858369232362487
Recall:  0.7601579134137686
F1 Score:  0.7220533928633788
####################################


-0.7220533928633788

In [47]:
gbm = GradientBoostingClassifier(n_estimators=400, learning_rate= 0.3, max_depth=7,random_state=42)
evaluate(gbm, X_pca_train,y_pca_train,X_pca_test,y_pca_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7332771469127041
Precision:  0.7396226853763646
Recall:  0.7332771469127041
F1 Score:  0.7094529903040842
####################################


-0.7094529903040842

### Tree Dataset

In [121]:
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7488023420865862
Precision:  0.8113205003485532
Recall:  0.7488023420865862
F1 Score:  0.7036452715792411
####################################


-0.7036452715792411

In [ ]:
# Define the search space for hyperparameters
gbm_search_space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400]),
    'max_depth': hp.choice('max_depth', [3, 5, 7, 9]),
    'learning_rate': hp.choice('learning_rate', [0.01, 0.05, 0.1, 0.3]),
    
    
}

# Define the objective function for optimizing F1 score
def objective_func_gbm(params):
    gbm = GradientBoostingClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        
        random_state=42
    )
    

    return train_model(gbm,X_tree_train,y_tree_train)

In [ ]:
gbm2_best = best_params(objective_func_gbm2, gbm_search_space)

100%|██████████| 50/50 [4:52:47<00:00, 351.34s/trial, best loss: -0.9895762709813092]  
Best Parameters: {'learning_rate': 0.03523010317024397, 'max_depth': 10.0, 'min_samples_leaf': 0.10046592387004868, 'min_samples_split': 0.10655581077489396, 'n_estimators': 350.0, 'subsample': 0.9865122012440967}


In [129]:
gbm = GradientBoostingClassifier(n_estimators=350, learning_rate= 0.03523010317024397, max_depth=10.0,min_samples_leaf=0.10046592387004868, min_samples_split= 0.10655581077489396, subsample=0.9865122012440967, random_state=0)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7341199432221434
Precision:  0.796853127179646
Recall:  0.7341199432221434
F1 Score:  0.6930681907940133
####################################


-0.6930681907940133

In [78]:
gbm = GradientBoostingClassifier(n_estimators=400, learning_rate= 0.3, max_depth=7,random_state=42)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7687189496096523
Precision:  0.8092477860316887
Recall:  0.7687189496096523
F1 Score:  0.7328192629980853
####################################


-0.7328192629980853

In [79]:
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate= 0.01, max_depth=9,random_state=42)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7565205819730305
Precision:  0.799188651182008
Recall:  0.7565205819730305
F1 Score:  0.7084699803290744
####################################


-0.7084699803290744

In [80]:
gbm = GradientBoostingClassifier(n_estimators=300, learning_rate= 0.1, max_depth=7,random_state=42)
evaluate(gbm, X_tree_train,y_tree_train,X_tree_test,y_tree_test)

Model:  GradientBoostingClassifier
Accuracy:  0.7609119943222143
Precision:  0.8092455602340636
Recall:  0.7609119943222143
F1 Score:  0.7160194927393677
####################################


-0.7160194927393677